## 필요한 라이브러리 설치


In [ ]:
!pip install pytorch-lightning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 6.4 MB/s 
     |████████████████████████████████| 125 kB 97.0 MB/s 
     |████████████████████████████████| 512 kB 83.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.0 MB/s 
     |████████████████████████████████| 7.6 MB 42.6 MB/s 
     |████████████████████████████████| 182 kB 81.7 MB/s 


## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 학습에서 사용되는 라이브러리 호출

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score

import pytorch_lightning as pl

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from transformers import AutoModel, AutoTokenizer

from torchmetrics.functional import accuracy,f1_score

import warnings
warnings.filterwarnings(action='ignore') 

## 반복적인 검증을 위해서 시드 값 고정

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(428)

## 학습 데이터 및 테스트 데이터 호출

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/test.csv")

## 학습 데이터 데이터 프레임 확인

학습 데이터에서는 테스트 데이터에 없는 `유형, 극성, 시제, 확실성, label` 데이터가 존재하는 것을 확인할 수 있습니다. 이를 이용한 학습을 수행하고자 합니다.

In [ ]:
train_df.head()

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [ ]:
test_df.head()

,ID,문장
0,TEST_0000,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정..."
1,TEST_0001,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다."
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...


## `유형, 극성, 시제, 확실성, label` 항목에 대해서 개별적인 라벨 인코딩을 적용

라벨 인코딩을 적용하게 되면 학습 데이터에서 보지 못했던 값에 대해서는 변환 시키지 못합니다. 이를 주의해야 합니다. 

In [ ]:
type_le = preprocessing.LabelEncoder()
train_df['유형_라벨인코딩'] = type_le.fit_transform(train_df['유형'])

polarity_le = preprocessing.LabelEncoder()
train_df['극성_라벨인코딩'] = polarity_le.fit_transform(train_df['극성'])

tense_le = preprocessing.LabelEncoder()
train_df['시제_라벨인코딩'] = tense_le.fit_transform(train_df['시제'])

certainty_le = preprocessing.LabelEncoder()
train_df['확실성_라벨인코딩'] = certainty_le.fit_transform(train_df['확실성'])

label_le = preprocessing.LabelEncoder()
train_df['label_라벨인코딩'] = label_le.fit_transform(train_df['label'])

## 각 항목에 대해서 변환된 클래스 종류

* 유형 -> 4개의 클래스가 존재
* 극성 -> 3개의 클래스가 존재
* 시제 -> 3개의 클래스가 존재
* 확실성 -> 2개의 클래스가 존재
* label -> 64개의 클래스가 존재

모델의 돌려보기 이전에는 단순히 유형 분류가 가장 어렵다고 생각된다. 반대로, 시제 분류는 모델 입장에서는 평의해 보일 것으로 추측합니다.

In [ ]:
print(type_le.classes_, " -> ", len(type_le.classes_))
print(polarity_le.classes_, " -> ", len(polarity_le.classes_))
print(tense_le.classes_, " -> ", len(tense_le.classes_))
print(certainty_le.classes_, " -> ", len(certainty_le.classes_))

['대화형' '사실형' '예측형' '추론형']  ->  4
['긍정' '미정' '부정']  ->  3
['과거' '미래' '현재']  ->  3
['불확실' '확실']  ->  2


## 라벨 인코딩이 적용된 학습 데이터의 모습

라벨 인코딩이 정상적으로 적용된 것을 확인할 수 있습니다.

In [ ]:
train_df.head()

,ID,문장,유형,극성,시제,확실성,label,유형_라벨인코딩,극성_라벨인코딩,시제_라벨인코딩,확실성_라벨인코딩,label_라벨인코딩
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,1,0,2,1,21
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,0,0,1,17
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,1,0,1,1,19
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,0,0,1,17
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,1,0,2,1,21


## 라벨 인코딩이 정상적으로 작동하는지를 확인

In [ ]:
label_le.transform(["사실형-긍정-현재-확실"])

array([21])

## 이후 모델 평가를 위해서 사용하는 사전 자료형

모델 학습시에 유형x극성x시제x확실성 조합에서 존재하지 않는 조합이 생성될 수 있습니다. 따라서, 존재하지 않는 조합에 대해서는 평가를 할 수 없기 때문에, `None`을 주어서 64로 변환한다.

In [ ]:
label_to_number = {x:idx for idx, x in enumerate(label_le.classes_)}
label_to_number['None'] = 64

In [ ]:
label_to_number

{'대화형-긍정-과거-불확실': 0,
 '대화형-긍정-과거-확실': 1,
 '대화형-긍정-미래-불확실': 2,
 '대화형-긍정-미래-확실': 3,
 '대화형-긍정-현재-불확실': 4,
 '대화형-긍정-현재-확실': 5,
 '대화형-미정-과거-불확실': 6,
 '대화형-미정-과거-확실': 7,
 '대화형-미정-미래-불확실': 8,
 '대화형-미정-미래-확실': 9,
 '대화형-미정-현재-불확실': 10,
 '대화형-부정-과거-불확실': 11,
 '대화형-부정-과거-확실': 12,
 '대화형-부정-미래-확실': 13,
 '대화형-부정-현재-불확실': 14,
 '대화형-부정-현재-확실': 15,
 '사실형-긍정-과거-불확실': 16,
 '사실형-긍정-과거-확실': 17,
 '사실형-긍정-미래-불확실': 18,
 '사실형-긍정-미래-확실': 19,
 '사실형-긍정-현재-불확실': 20,
 '사실형-긍정-현재-확실': 21,
 '사실형-미정-과거-확실': 22,
 '사실형-미정-미래-불확실': 23,
 '사실형-미정-미래-확실': 24,
 '사실형-미정-현재-불확실': 25,
 '사실형-미정-현재-확실': 26,
 '사실형-부정-과거-불확실': 27,
 '사실형-부정-과거-확실': 28,
 '사실형-부정-미래-불확실': 29,
 '사실형-부정-미래-확실': 30,
 '사실형-부정-현재-불확실': 31,
 '사실형-부정-현재-확실': 32,
 '예측형-긍정-과거-불확실': 33,
 '예측형-긍정-과거-확실': 34,
 '예측형-긍정-미래-불확실': 35,
 '예측형-긍정-미래-확실': 36,
 '예측형-긍정-현재-불확실': 37,
 '예측형-긍정-현재-확실': 38,
 '예측형-미정-과거-확실': 39,
 '예측형-미정-미래-불확실': 40,
 '예측형-미정-미래-확실': 41,
 '예측형-미정-현재-불확실': 42,
 '예측형-미정-현재-확실': 43,
 '예측형-부정-과거-확실': 44,
 '예측형-부정-미래-불확실': 45,
 '예측형-부정-현재-불확실': 46,


## Dataset & TextClassificationCollator 선언

* Dataset은 일반적으로 pytorch에서 사용하는 Dataset 그대로 입니다.
* TextClassificationCollator는 모델 입력이전에 형태를 변화시켜 주기 위해서 사용합니다.

학습시에 사용되는 Dataset과 테스트 데이터에서 사용되는 Dataset에는 일부 차이가 존재합니다. 왜냐하면 테스트 데이터에서는 별도의 유형, 시제, 극성, 확실성, 라벨 항목등이 없기 때문입니다.

In [ ]:
class TextClassificationCollator():
    def __init__(self, model_name, train=False):
        self.train = train
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=512, truncation=True, padding = 'max_length', return_tensors = "pt")

    def __call__(self, samples):
        # 🤗 아래와 반대임
        # 수정하기 귀찮아서 그대로 사용 😏
        # 수정 방법은 그냥 반대로 조건문을 넣거나 or !self.train 하면 됩니다.
        if self.train:
            context = self.tokenizer(samples, padding = True, truncation = True, return_tensors = "pt")
            return context
        else:
            # print(samples)
            # print(len(samples[1]))
            x, y, y_type, y_polarity, y_tense, y_certainty = [[] for _ in range(6)]

            for idx in range(len(samples)):
                x.append(samples[idx][0])
                y.append(samples[idx][1])
                y_type.append(samples[idx][2])
                y_polarity.append(samples[idx][3])
                y_tense.append(samples[idx][4])
                y_certainty.append(samples[idx][5])

            context = self.tokenizer(x, padding = True, truncation = True, return_tensors = "pt")
            return context, torch.LongTensor(y), torch.LongTensor(y_type), torch.LongTensor(y_polarity), torch.LongTensor(y_tense), torch.LongTensor(y_certainty)

class BaseDataset(Dataset):
    def __init__(self, df):
        self.sentences = df['문장']
        self.labels = torch.LongTensor(df['label_라벨인코딩'])
        self.type_labels = torch.LongTensor(df['유형_라벨인코딩'])
        self.polarity_labels = torch.LongTensor(df['극성_라벨인코딩'])
        self.tense_labels = torch.LongTensor(df['시제_라벨인코딩'])
        self.certainty_labels = torch.LongTensor(df['확실성_라벨인코딩'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sentences[idx], self.labels[idx], self.type_labels[idx], self.polarity_labels[idx], self.tense_labels[idx], self.certainty_labels[idx]

class BaseDatasetTest(Dataset):
    def __init__(self, df):
        self.sentences = df['문장']

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [ ]:
dataset = BaseDataset(train_df)
test_dataset = BaseDatasetTest(test_df)

In [ ]:
import bisect
import functools
import warnings
from torch import default_generator, randperm
from torch._utils import _accumulate
from typing import (
    Callable,
    Dict,
    Generic,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    Tuple,
    TypeVar,
)

T_co = TypeVar('T_co', covariant=True)
T = TypeVar('T')

UNTRACABLE_DATAFRAME_PIPES = ['batch',  # As it returns DataChunks
                              'groupby',   # As it returns DataChunks
                              '_dataframes_as_tuples',  # As it unpacks DF
                              'trace_as_dataframe',  # As it used to mark DF for tracing
                              ]
class Subset(Dataset[T_co]):
    r"""
    Subset of a dataset at specified indices.

    Args:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    dataset: Dataset[T_co]
    indices: Sequence[int]

    def __init__(self, dataset: Dataset[T_co], indices: Sequence[int]) -> None:
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        if isinstance(idx, list):
            return self.dataset[[self.indices[i] for i in idx]]
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)


def select_split(dataset: Dataset[T], indices) -> List[Subset[T]]:
    indices = indices
    return Subset(dataset, indices)

## 학습을 위해서 사용되는 Lightning 모듈

Lightning 모듈에는 training_step, validation_step, preict_step 등이 존재합니다.
* train_step 같은 경우는 학습에서 사용되는 부분입니다.
* validation_step 같은 경우는 검증을 위해서 사용되는 부분입니다.
* preict_step 같은 경우는 최종적인 test 데이터에 대해서 예측을 수행합니다.

학습에서는 validation에서 f1score가 가장 높은 모델을 자동 저장하도록 합니다. 

In [ ]:
# type -> 유형
# polarity -> 극성
# tense -> 시제
# certainty -> 확실성
class ClassifySentenceType(pl.LightningModule):

    def __init__(self, model_name="klue/roberta-small"):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)

        self.type_classification = nn.Linear(768, 4)
        self.polarity_classification = nn.Linear(768, 3)
        self.tense_classification = nn.Linear(768, 3)
        self.certainty_classification = nn.Linear(768, 2)

    def forward(self, x):
        tokenizers = x
        context = self.model(input_ids = tokenizers['input_ids'], attention_mask = tokenizers['attention_mask'])
        context = context[1]

        type_hat = self.type_classification(context)
        polarity_hat = self.polarity_classification(context)
        tense_hat = self.tense_classification(context)
        certainty_hat = self.certainty_classification(context)

        return {
            'type':type_hat,
            'polarity':polarity_hat,
            'tense':tense_hat,
            'certainty':certainty_hat
        }

    def training_step(self, batch, batch_idx):
        tokenizers, y, y_type, y_polarity, y_tense, y_certainty = batch

        context = self.model(input_ids = tokenizers['input_ids'], attention_mask = tokenizers['attention_mask'])
        context = context[1]
        type_hat = self.type_classification(context)
        polarity_hat = self.polarity_classification(context)
        tense_hat = self.tense_classification(context)
        certainty_hat = self.certainty_classification(context)

        type_loss = F.cross_entropy(type_hat, y_type)
        polarity_loss = F.cross_entropy(polarity_hat, y_polarity)
        tense_loss = F.cross_entropy(tense_hat, y_tense)
        certainty_loss = F.cross_entropy(certainty_hat, y_certainty)

        type_ = type_le.inverse_transform(type_hat.argmax(dim = -1).tolist())
        polarity_ = polarity_le.inverse_transform(polarity_hat.argmax(dim = -1).tolist())
        tense_ = tense_le.inverse_transform(tense_hat.argmax(dim = -1).tolist())
        certainty_ = certainty_le.inverse_transform(certainty_hat.argmax(dim = -1).tolist())

        labels = []
        for idx in range(type_hat.shape[0]):
            temp = f"{type_[idx]}-{polarity_[idx]}-{tense_[idx]}-{certainty_[idx]}"
            try:
                labels.append(label_to_number[temp])
            except KeyError as e:
                labels.append(label_to_number['None'])

        f1 = f1_score(torch.tensor(labels).to(y.device), y, task='multiclass', num_classes=len(label_to_number))
        # print("😂😂😂")

        loss = (type_loss + polarity_loss + tense_loss + certainty_loss) / 4

        metrics = {'train_loss':loss, 'train_f1score':f1}
        # metrics = {'train_loss':loss}
        self.log_dict(metrics, prog_bar = True)
        return {
            "loss":loss
        }

    def validation_step(self, batch, batch_idx):
        tokenizers, y, y_type, y_polarity, y_tense, y_certainty = batch

        context = self.model(input_ids = tokenizers['input_ids'], attention_mask = tokenizers['attention_mask'])
        context = context[1]
        type_hat = self.type_classification(context)
        polarity_hat = self.polarity_classification(context)
        tense_hat = self.tense_classification(context)
        certainty_hat = self.certainty_classification(context)

        type_loss = F.cross_entropy(type_hat, y_type)
        polarity_loss = F.cross_entropy(polarity_hat, y_polarity)
        tense_loss = F.cross_entropy(tense_hat, y_tense)
        certainty_loss = F.cross_entropy(certainty_hat, y_certainty)

        type_ = type_le.inverse_transform(type_hat.argmax(dim = -1).tolist())
        polarity_ = polarity_le.inverse_transform(polarity_hat.argmax(dim = -1).tolist())
        tense_ = tense_le.inverse_transform(tense_hat.argmax(dim = -1).tolist())
        certainty_ = certainty_le.inverse_transform(certainty_hat.argmax(dim = -1).tolist())

        labels = []
        for idx in range(type_hat.shape[0]):
            temp = f"{type_[idx]}-{polarity_[idx]}-{tense_[idx]}-{certainty_[idx]}"
            try:
                labels.append(label_to_number[temp])
            except KeyError as e:
                labels.append(label_to_number['None'])

        f1 = f1_score(torch.tensor(labels).to(y.device), y, task='multiclass', num_classes=len(label_to_number))
        # print("😂😂😂")

        loss = (type_loss + polarity_loss + tense_loss + certainty_loss) / 4

        metrics = {'val_loss':loss, 'val_f1score':f1}
        self.log_dict(metrics, prog_bar = True)
        return {
            "val_loss":loss,
            "val_f1score":f1
        }

    def preict_step(self, batch, batch_idx, dataloader_idx=0):
        tokenizers = batch
        context = self.model(input_ids = tokenizers['input_ids'], attention_mask = tokenizers['attention_mask'])
        context = context[1]
        
        type_hat = self.type_classification(context)
        polarity_hat = self.polarity_classification(context)
        tense_hat = self.tense_classification(context)
        certainty_hat = self.certainty_classification(context)
        
        return {
            'type':type_hat,
            'polarity':polarity_hat,
            'tense':tense_hat,
            'certainty':certainty_hat
        }

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return optimizer

## 교차 검증을 위해서 k-fold 값을 5로 설정함

* 초기 학습에서 오류가 있어서 1번째 모델 같은 경우는 별도 코랩 창에서 학습을 수행했습니다.

모델은 "klue/roberta-base"를 사용했습니다. 이 모델을 사용한 이유는 제가 대회 참여한 기간이 하루밖에 되지 않아서, 큰 모델을 사용하게 되면 학습 시간이 부족하다고 판단되어서 작은 모델을 사용했습니다.

In [ ]:
from pytorch_lightning.callbacks import RichProgressBar, EarlyStopping, ModelCheckpoint


skf = StratifiedKFold(n_splits = 5, random_state=428, shuffle=True)

model_name = "klue/roberta-base"

for idx, (train_index, valid_index) in enumerate(skf.split(dataset, train_df['label_라벨인코딩'])):
    print(idx)
    if idx == 0:
        continue
    dirpath = f"/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold{idx}"
    checkpoint_callback = ModelCheckpoint(
    dirpath=dirpath,
    save_top_k =5,
    filename='{epoch}-{step}-{train_f1score}-{val_f1score:.2f}',
    verbose=True,
    monitor='val_f1score',
    mode='max'
    )
    
    train_dataset = select_split(dataset, train_index)
    valid_dataset = select_split(dataset, valid_index)
    train_dataloader = DataLoader(train_dataset, batch_size = 32, collate_fn=TextClassificationCollator(model_name=model_name), shuffle = True)
    valid_dataloader = DataLoader(valid_dataset, batch_size = 32, collate_fn=TextClassificationCollator(model_name=model_name), shuffle = False)
    
    bert = ClassifySentenceType(model_name=model_name)
    trainer = pl.Trainer(max_epochs = 9, accelerator="gpu", accumulate_grad_batches = 1, 
                         callbacks=[EarlyStopping('val_f1score', patience = 6, mode='max', verbose = True), checkpoint_callback])
    trainer.fit(bert, train_dataloader, valid_dataloader)
    del trainer, bert, train_dataset, valid_dataset, train_dataloader, valid_dataloader
    torch.cuda.empty_cache() 

0
1


Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 6, global step 2898: 'val_f1score' was not in top 5


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 3312: 'val_f1score' was not in top 5


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 3726: 'val_f1score' was not in top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=9` reached.


## 테스트 데이터에 대한 예측 수행

In [ ]:
model_name = "klue/roberta-base"
test_dataloader = DataLoader(test_dataset, batch_size = 40, collate_fn=TextClassificationCollator(model_name=model_name, train = True), shuffle = False)

In [ ]:
models = [
    "/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold0/epoch=11-step=3972-train_f1score=1.0-val_f1score=0.99.ckpt",
    "/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold1/epoch=2-step=1242-train_f1score=1.0-val_f1score=0.76.ckpt",
    "/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold2/epoch=5-step=2484-train_f1score=0.8823529481887817-val_f1score=0.76.ckpt",
    "/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold3/epoch=4-step=2070-train_f1score=0.8823529481887817-val_f1score=0.75.ckpt",
    "/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/fold4/epoch=5-step=2484-train_f1score=0.8823529481887817-val_f1score=0.75.ckpt"
]

preds = []
for i in range(5):
    temp = torch.load(models[i])
    m = ClassifySentenceType(model_name = "klue/roberta-base")
    m.load_state_dict(temp['state_dict'])
    trainer = pl.Trainer(accelerator="gpu")
    preds.append(trainer.predict(m, test_dataloader))


Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Predicting: 0it [00:00, ?it/s]

In [ ]:
num = 0
one_type_hat = preds[num][0]['type']
one_polarity_hat = preds[num][0]['polarity']
one_tense_hat = preds[num][0]['tense']
one_certainty_hat = preds[num][0]['certainty']

for i in range(len(preds[num])):
    if i == 0:
        continue
    else:
        one_type_hat = torch.vstack((one_type_hat, preds[num][i]['type']))
        one_polarity_hat = torch.vstack((one_polarity_hat, preds[num][i]['polarity']))
        one_tense_hat = torch.vstack((one_tense_hat, preds[num][i]['tense']))
        one_certainty_hat = torch.vstack((one_certainty_hat, preds[num][i]['certainty']))

num = 1
two_type_hat = preds[num][0]['type']
two_polarity_hat = preds[num][0]['polarity']
two_tense_hat = preds[num][0]['tense']
two_certainty_hat = preds[num][0]['certainty']

for i in range(len(preds[num])):
    if i == 0:
        continue
    else:
        two_type_hat = torch.vstack((two_type_hat, preds[num][i]['type']))
        two_polarity_hat = torch.vstack((two_polarity_hat, preds[num][i]['polarity']))
        two_tense_hat = torch.vstack((two_tense_hat, preds[num][i]['tense']))
        two_certainty_hat = torch.vstack((two_certainty_hat, preds[num][i]['certainty']))

num = 2
three_type_hat = preds[num][0]['type']
three_polarity_hat = preds[num][0]['polarity']
three_tense_hat = preds[num][0]['tense']
three_certainty_hat = preds[num][0]['certainty']

for i in range(len(preds[num])):
    if i == 0:
        continue
    else:
        three_type_hat = torch.vstack((three_type_hat, preds[num][i]['type']))
        three_polarity_hat = torch.vstack((three_polarity_hat, preds[num][i]['polarity']))
        three_tense_hat = torch.vstack((three_tense_hat, preds[num][i]['tense']))
        three_certainty_hat = torch.vstack((three_certainty_hat, preds[num][i]['certainty']))

num = 3
four_type_hat = preds[num][0]['type']
four_polarity_hat = preds[num][0]['polarity']
four_tense_hat = preds[num][0]['tense']
four_certainty_hat = preds[num][0]['certainty']

for i in range(len(preds[num])):
    if i == 0:
        continue
    else:
        four_type_hat = torch.vstack((four_type_hat, preds[num][i]['type']))
        four_polarity_hat = torch.vstack((four_polarity_hat, preds[num][i]['polarity']))
        four_tense_hat = torch.vstack((four_tense_hat, preds[num][i]['tense']))
        four_certainty_hat = torch.vstack((four_certainty_hat, preds[num][i]['certainty']))

num = 4
five_type_hat = preds[num][0]['type']
five_polarity_hat = preds[num][0]['polarity']
five_tense_hat = preds[num][0]['tense']
five_certainty_hat = preds[num][0]['certainty']

for i in range(len(preds[num])):
    if i == 0:
        continue
    else:
        five_type_hat = torch.vstack((five_type_hat, preds[num][i]['type']))
        five_polarity_hat = torch.vstack((five_polarity_hat, preds[num][i]['polarity']))
        five_tense_hat = torch.vstack((five_tense_hat, preds[num][i]['tense']))
        five_certainty_hat = torch.vstack((five_certainty_hat, preds[num][i]['certainty']))

In [ ]:
final_type_hat = (one_type_hat + two_type_hat + three_type_hat + four_type_hat + five_type_hat)
final_polarity_hat = (one_polarity_hat + two_polarity_hat + three_polarity_hat + four_polarity_hat + five_polarity_hat)
final_tense_hat = (one_tense_hat + two_tense_hat + three_tense_hat + four_tense_hat + five_tense_hat)
final_certainty_hat = (one_certainty_hat + two_certainty_hat + three_certainty_hat + four_certainty_hat+five_certainty_hat)

In [ ]:
print(final_type_hat.shape)
print(final_polarity_hat.shape)
print(final_tense_hat.shape)
print(final_certainty_hat.shape)

torch.Size([7090, 4])
torch.Size([7090, 3])
torch.Size([7090, 3])
torch.Size([7090, 2])


In [ ]:
type_ = type_le.inverse_transform(final_type_hat.argmax(dim = -1).tolist())
polarity_ = polarity_le.inverse_transform(final_polarity_hat.argmax(dim = -1).tolist())
tense_ = tense_le.inverse_transform(final_tense_hat.argmax(dim = -1).tolist())
certainty_ = certainty_le.inverse_transform(final_certainty_hat.argmax(dim = -1).tolist())

In [ ]:
labels = []
for idx in range(type_.shape[0]):
    temp = f"{type_[idx]}-{polarity_[idx]}-{tense_[idx]}-{certainty_[idx]}"
    labels.append(temp)

In [ ]:
submit = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/sample_submission.csv")
submit['label'] = labels

In [ ]:
submit.head()

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실


In [ ]:
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/DACON/문장_유형_분류_AI_경진대회/baseline_submit.csv', index=False)